In [1]:
import numpy as np
import tensorflow as tf
from itertools import product
import random
#import tfnumpy as tfnp

In [2]:
x1 = np.array([[1, 0], [0, 1], [2, -1.4]], dtype=np.float32)
x2 = np.array([[0, -1], [1, 0], [-1.5, 0.6]], dtype=np.float32)
x = np.zeros((3, 2, 2))
x[:, :, 0] = x1
x[:, :, 1] = x2

# Using numpy first

In [3]:
a = np.random.uniform(size=x.shape[2])
b = np.random.uniform(size=x.shape[2])

In [4]:
x[:, :, 0]*a[0]+x[:, :, 1]*a[1]

array([[ 0.09112611, -0.59562184],
       [ 0.59562184,  0.09112611],
       [-0.71118054,  0.22979657]])

In [5]:
Ta = sum(x[:, :, i]*a[i] for i in range(x.shape[2]))
Ta

array([[ 0.09112611, -0.59562184],
       [ 0.59562184,  0.09112611],
       [-0.71118054,  0.22979657]])

In [6]:
Tb = sum(x[:, :, i]*b[i] for i in range(x.shape[2]))
Tb

array([[ 0.35302937, -0.8967192 ],
       [ 0.8967192 ,  0.35302937],
       [-0.63902006,  0.04379043]])

In [7]:
eigvals_a, eigvecs_a = np.linalg.eig(np.matmul(Ta, np.conjugate(Tb.T)))

In [8]:
eigvals_b, eigvecs_b = np.linalg.eig(np.conjugate(np.transpose(np.matmul(np.conjugate(Tb.T), Ta))))

In [9]:
eigvals_a

array([ 9.48682150e-01,  6.48390837e-01, -1.03043292e-16])

In [10]:
eigvals_b

array([0.94868215, 0.64839084])

In [11]:
# pair up eigenvalues of Ta and Tb
idx_pairs = []
tol = 1e-5

for i, eigval_a in enumerate(eigvals_a):
    for j, eigval_b in enumerate(eigvals_b):
        if abs(eigval_a - eigval_b) < tol:
            idx_pairs += [(i, j)]
            break

idx_pairs

[(0, 0), (1, 1)]

In [14]:
nbcomp = len(idx_pairs)
A = np.zeros((nbcomp*x.shape[2], nbcomp*x.shape[2]))
B = np.zeros(nbcomp*x.shape[2])
eqidx = 0
for k in range(x.shape[2]):
    for i, j in random.choices(list(product(range(x.shape[0]), range(x.shape[1]))), k=nbcomp):
        #print("{} {} {}".format(i, j, k))
        B[eqidx] = x[i, j, k]
        for ck in range(nbcomp):
            A[eqidx, ck*nbcomp+k] = eigvecs_a[idx_pairs[ck][0], i]*eigvecs_b[idx_pairs[ck][1], j]
        eqidx += 1

In [17]:
sol = np.linalg.solve(A, B)
sol

array([ 5.46398458, -1.25720609, -3.21753026,  4.2376251 ])

In [20]:
eigvecs_c = sol.reshape((nbcomp, x.shape[2]), order='F')
eigvecs_c

array([[ 5.46398458, -3.21753026],
       [-1.25720609,  4.2376251 ]])

In [34]:
# verfication
xh = np.zeros(x.shape)
for i, j, k in product(range(x.shape[0]), range(x.shape[1]), range(x.shape[2])):
    xh[i, j, k] = sum([eigvecs_a[alpha, i]*eigvecs_b[alpha, j]*eigvecs_c[alpha, k] for alpha in range(nbcomp)])

In [35]:
xh

array([[[ 0.19853161, -1.5467626 ],
        [-0.73690431,  2.76013739]],

       [[ 2.38614556, -0.03303901],
        [-0.10405237, -2.17092161]],

       [[ 1.41715524, -1.87432896],
        [-0.93658567,  2.24318313]]])

In [36]:
x

array([[[ 1.        ,  0.        ],
        [ 0.        , -1.        ]],

       [[ 0.        ,  1.        ],
        [ 1.        ,  0.        ]],

       [[ 2.        , -1.5       ],
        [-1.39999998,  0.60000002]]])